# Section 0

QPM: Assignment 4

DENG Yunyun <br>
LI Jiaxin <br>
SBAI Ilyas <br>
ZHOU Zhichen <br>

Note: NA

# Initial step to prepare the data (4.0)
We simply replicate the data retrieving process from last assignment and store the dataframes into sql database

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import sqlite3

# Date range
start_date = "2000-01-01"
end_date = "2022-12-31"

# We retrieve the list of tickers of the S&P500 from Wikipedia
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tic = (pd.read_html(URL)[0]['Symbol'].str.replace('.', '-', regex=True) .unique().tolist())

# Importing the  data using the download function from the yf package
data = (yf.download(tickers=tic,
                    start=start_date,
                    end=end_date,
                   progress=False)
        .melt(ignore_index=False, 
              var_name=["variable", "symbol"])
        .reset_index()
        .pivot(index=["Date", "symbol"], 
               columns="variable", 
               values="value")
        .reset_index()
        .rename(columns = {"Date": "date",
                   "Open": "open",
                   "High": "high",
                   "Low": "low",
                   "Close": "close",
                   "Adj Close": "adjusted",
                   "Volume": "volume"
                  }))

# extract the price data
tic_price = (data.get(["symbol", "date", "adjusted"]).pivot(index = 'date', columns='symbol', values = 'adjusted')
             .dropna(axis=1, how='all'))
tic_price = tic_price.loc[:, (tic_price.isna().sum(axis=0) <= 100)].reset_index()

#save the data in a database for future reference
SnP500 = sqlite3.connect(database="/Users/lijiaxin/Downloads/M2/S2/QPM/snp500.sqlite")

(data.to_sql(name="SnP500_daily", con=SnP500, if_exists="replace",index = False))

(tic_price.to_sql(name='SnP500_daily_adjusted_price', con=SnP500, if_exists="replace",index = False))


2 Failed downloads:
['KVUE', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 946702800, endDate = 1672462800")


5787

Access data stored in the database

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import sqlite3

#Establish the connection to the SQLite database
SnP500 = sqlite3.connect(database="/Users/lijiaxin/Downloads/M2/S2/QPM/snp500.sqlite")

#Execute the query to fetch the data
stock_price = (pd.read_sql_query(
        sql="SELECT * FROM SnP500_daily_adjusted_price",
        con=SnP500))

#Add date time index to the data
stock_price['date'] = pd.to_datetime(stock_price['date'])
stock_price = stock_price.set_index('date')

#Select the 10 companies
com_list = ['MMM','AOS','ABT','ADM','ADBE','ADP','AES','AFL','A','AKAM']
pt = stock_price.loc[:, com_list]

#Compute the monthly returns
ret_daily=pt.pct_change().dropna(axis=0)
ret_monthly = ret_daily.resample('1M').agg(lambda x: (1 + x).prod() - 1)
ret_monthly.head()

,MMM,AOS,ABT,ADM,ADBE,ADP,AES,AFL,A,AKAM
date,,,,,,,,,,
2000-01-31,-0.007947,-0.084087,-0.065070,-0.020834,-0.160153,-0.087740,0.105172,-0.029330,-0.080729,-0.224514
2000-02-29,-0.051976,-0.126583,0.013437,-0.140034,0.852440,-0.081687,0.046022,-0.156654,0.569404,0.048670
2000-03-31,0.004252,0.043479,0.066287,0.024845,0.091567,0.109849,-0.060403,0.246154,0.001204,-0.384450
2000-04-30,-0.021877,0.149574,0.097843,-0.036364,0.086468,0.115285,0.142063,0.071331,-0.147836,-0.385154
2000-05-31,-0.003685,0.021277,0.058537,0.207109,-0.069251,0.022068,-0.029882,0.060746,-0.169253,-0.324905


In [3]:
#save the daily price for ten companies in the database
(pt.to_sql(name="TenCompanies_price_daily", con=SnP500, if_exists="replace",index = False))

#save the monthly return for ten companies in the database
(ret_monthly.to_sql(name="TenCompanies_ret_monthly", con=SnP500, if_exists="replace",index = False))

276

# Section 1

We first define the necessary data: the estimation window ```T_est```, the vector of monthly mean returns ```mu```, the variance-covariance matrix of annualised returns ```V``` and its inverse ```Vinverse```. <br>
We use the data from the first 60 days.

In [4]:
#Set the first 60 months to be the estimation window
T_est = 60
sp = ret_monthly.iloc[0:T_est,:]

#Define a column vector of ones
ones = np.ones((10,1))

#Column vector of monthly mean returns
mu = sp.mean().to_numpy().reshape(-1,1)

#Variance-covariance matrix
V = sp.cov().to_numpy()

#Define inverse of the variance-covariance matrix
Vinverse = np.linalg.inv(V)

We then define the function to calculate the optimal weights for mean-variance portfolio where risk free asset is available.

In [5]:
#Define solution function when risk-free asset is available
def sol_MVP (mu, mu_targ, Rf, Vinverse):
    term1Numerator = (mu_targ - Rf)
    term1Denominator = (mu - Rf*ones).T@Vinverse@(mu-Rf*ones)
    term1 = term1Numerator/term1Denominator
    term2 = Vinverse@(mu - Rf*ones)
    weightsWRf = term1*term2
    portVol = np.sqrt(weightsWRf.T@V@weightsWRf)
    
    return weightsWRf, portVol

In [6]:
#Trial: set expected monthly return to be 10%

w_MVP = sol_MVP(mu, 0.1, 0, Vinverse)[0]
vol_MVP = sol_MVP(mu, 0.1, 0, Vinverse)[1]

print('Optimal weights for mean variance portfolio are: ','\n', w_MVP)
print("Volatility of Optimal mean variance portfolio's return is: ", vol_MVP[0,0])

Optimal weights for mean variance portfolio are:  
 [[ 1.43846207]
 [ 0.66512242]
 [ 0.92204306]
 [ 1.71612904]
 [ 1.16181618]
 [-1.2897939 ]
 [ 0.23373574]
 [ 0.14405685]
 [-0.73955557]
 [-0.04498322]]
Volatility of Optimal mean variance portfolio's return is:  0.24983198532787276


We also calculate the optimal weights for Global Minimum Variance Portfolio based on the following formula:

$$
w = \frac{V^{-1}1_N}{1^{T}_{N}V^{-1}1_N}
$$

In [7]:
w_GMV = (Vinverse@ones)/(ones.T@Vinverse@ones)
vol_GMV = np.sqrt(w_GMV.T@V@w_GMV)

print('Optimal weights for global minimum variance portfolio are: ','\n', w_GMV)
print("Volatility of Optimal global minimum variance portfolio's return is: ", vol_GMV[0,0])

Optimal weights for global minimum variance portfolio are:  
 [[ 0.34792449]
 [ 0.10125196]
 [ 0.30909196]
 [ 0.04632376]
 [ 0.02444576]
 [ 0.08107936]
 [ 0.02619765]
 [ 0.07877339]
 [ 0.00909168]
 [-0.02418   ]]
Volatility of Optimal global minimum variance portfolio's return is:  0.04204581520058148


# Section 2

We use the rolling window $T^{est}=60$ to estimate the time series of optimal weights for both strategies by looping through the dates.

In [8]:
#set the target return for mean variance portfolio
mu_targ = 0.1

#create empty list to store the weights
w_MVP_list = []
w_GMV_list = []

#loop through the dates,
#at each date compute the weights for the two strategies based on the previous 60 days of monthly returns

for i in range (0,len(ret_monthly)-60):
    #calculate the metrics for each window
    sp_est = ret_monthly.iloc[i:T_est+i,:]
    mu_est = sp_est.mean().to_numpy().reshape(-1,1)
    V_est = sp_est.cov().to_numpy()
    Vinverse_est = np.linalg.inv(V_est)
    
    #calculate the weights 
    w_MVP_est = sol_MVP(mu_est, mu_targ,0, Vinverse_est)[0]    
    w_GMV_est = (Vinverse_est@ones)/(ones.T@Vinverse_est@ones)
    
    w_MVP_list.append(w_MVP_est)
    w_GMV_list.append(w_GMV_est)

#create the time series of weights by adding up the datetime index
w_MVP_ts = pd.Series(w_MVP_list, index=ret_monthly.index[60:276])
w_GMV_ts = pd.Series(w_GMV_list, index=ret_monthly.index[60:276])

In [9]:
w_GMV_ts

date
2005-01-31    [[0.3479244923649019], [0.10125195885207955], ...
2005-02-28    [[0.33732312912137663], [0.10545064711129269],...
2005-03-31    [[0.34041862290756114], [0.1154878157420604], ...
2005-04-30    [[0.30136794407810624], [0.1121930748474487], ...
2005-05-31    [[0.27107813076327786], [0.12930849419927473],...
                                    ...                        
2022-08-31    [[0.0033973834414433254], [-0.1826579171493633...
2022-09-30    [[-0.011828791063904814], [-0.1881856072654844...
2022-10-31    [[-0.014524870452502062], [-0.2048195381234758...
2022-11-30    [[-0.03153751387378908], [-0.22141978203170257...
2022-12-31    [[0.0032154622580793696], [-0.2437350386072495...
Freq: M, Length: 216, dtype: object

In [10]:
w_MVP_ts

date
2005-01-31    [[1.4384620712492464], [0.6651224205729019], [...
2005-02-28    [[1.2668292589411978], [0.6617258104500232], [...
2005-03-31    [[1.4599118678358225], [0.7390817745019078], [...
2005-04-30    [[1.618074890876644], [0.7584874860612336], [0...
2005-05-31    [[1.8555522577380195], [0.8772364741435219], [...
                                    ...                        
2022-08-31    [[-3.1170743614007512], [-0.6386374037178731],...
2022-09-30    [[-2.9245416362594834], [-0.6533473803700997],...
2022-10-31    [[-3.108338002419104], [-0.8175963501019322], ...
2022-11-30    [[-3.1373097961008924], [-0.5486031771302647],...
2022-12-31    [[-2.8909639993682648], [-0.5943076894383263],...
Freq: M, Length: 216, dtype: object

# Section 3

We calculate the out-of-sample returns for both strategies at time $t+1$ by multiplying the weights estimated from the information available until time $t$ by the stock returns at time $t+1$.

In [11]:
#create an empty list
ret_MVP_list = []
ret_GMV_list = []

for i in range (0,len(ret_monthly)-60):
    ret_MVP_est = (ret_monthly.iloc[T_est+i,:].to_numpy())@w_MVP_ts[i]
    ret_MVP_list.append(ret_MVP_est)
    
    ret_GMV_est = (ret_monthly.iloc[T_est+i,:].to_numpy())@w_GMV_ts[i]
    ret_GMV_list.append(ret_GMV_est)
    
#create the time series of monthly returns by adding up the datetime index 
ret_MVP_ts = pd.Series(ret_MVP_list, index=ret_monthly.index[60:])
ret_GMV_ts = pd.Series(ret_GMV_list, index=ret_monthly.index[60:])

In [12]:
ret_MVP_ts

date
2005-01-31     [0.08224024555581705]
2005-02-28     [0.06518040562571623]
2005-03-31     [0.23660342758555863]
2005-04-30     [-0.6353688797741495]
2005-05-31      [0.1538346598381014]
                       ...          
2022-08-31     [0.25053298376522015]
2022-09-30    [-0.08696599471892744]
2022-10-31     [0.06626802520457337]
2022-11-30     [0.34837822700920185]
2022-12-31    [-0.01821260167862898]
Freq: M, Length: 216, dtype: object

# Section 4

In [13]:
SR_MVP = np.sqrt(12)*np.mean(ret_MVP_ts) / np.std(ret_MVP_ts)
SR_GMV = np.sqrt(12)*np.mean(ret_GMV_ts) / np.std(ret_GMV_ts)

print('The Sharpe ratio for the mean-variance portfolio is:',SR_MVP[0],'\n')
print('The Sharpe ratio for the global minimum variance portfolio is:',SR_GMV[0], '\n')

if SR_MVP > SR_GMV:
    print('The mean-variance portfolio has higher sharpe ratio.')
else: 
    print('The global minimum variance portfolio has higher sharpe ratio.')

The Sharpe ratio for the mean-variance portfolio is: 0.4914436024627835 

The Sharpe ratio for the global minimum variance portfolio is: 0.8485161373122051 

The global minimum variance portfolio has higher sharpe ratio.
